In [3]:
from nba_api.stats.endpoints import playercareerstats
import matplotlib.pyplot as plt
from nba_api.stats.static import players
import seaborn as sns
import numpy as np
import pandas as pd
from json import loads, dumps
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
import time


pd.set_option("display.max_columns", None)

In [4]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
from nba_api.stats.endpoints import playergamelog
from helpfuncs import get_current_season

In [5]:
class player():
    def get_player_info(self, player_id:int) -> pd.DataFrame:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        info_df = player_info.get_data_frames()[0]
        return info_df
    
    def get_player_latest_performance(self, player_id:int) -> pd.Series | str:
        game_log = playergamelog.PlayerGameLog(
            player_id=player_id,
            season=get_current_season()
        )
        df = game_log.get_data_frames()[0]

        if df.empty:
            return "Player is not active this season or has not played any games."
        return df.iloc[0]

    def get_alltime_player_stats(self, player_id:int) -> pd.DataFrame:
        career = playercareerstats.PlayerCareerStats(player_id=player_id)
        career_df = career.get_data_frames()[0]

        return career_df

    def get_current_season_stats(self, player_id:int) -> pd.DataFrame:
        player_dashboard = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(
            player_id=player_id,
            season=get_current_season()
        )

        current_stats = player_dashboard.get_data_frames()[0]
        current_stats = current_stats[["GP", "MIN", "FG_PCT", "FG3_PCT", "FT_PCT", "REB", 
                                    "AST", "PTS", "BLK", "PLUS_MINUS"]]
        current_stats["PPG"] = round(current_stats["PTS"] / current_stats["GP"], 2)
        current_stats["APG"] = round(current_stats["AST"] / current_stats["GP"], 2)
        current_stats["RPG"] = round(current_stats["REB"] / current_stats["GP"], 2)
        current_stats["BPG"] = round(current_stats["BLK"] / current_stats["GP"], 2)
        current_stats["MPG"] = round(current_stats["MIN"] / current_stats["GP"], 2) 
        current_stats
        return current_stats


    def plot_stat_over_career(self, player_id, stat):
        player_name = players.find_player_by_id(player_id)
        career_df = self.get_alltime_player_stats(player_id=player_id)
        if stat not in career_df.columns:
            raise ValueError(f"Stat '{stat}' not found in career data.")
        
        print(player_name)
        plt.figure(figsize=(10, 6))
        sns.lineplot(data=career_df, x='SEASON_ID', y=stat, marker='o')
        plt.title(f"{player_name['full_name']}'s {stat} Over Career")
        plt.xlabel('Season')
        plt.ylabel(stat)
        plt.xticks(rotation=45)
        plt.grid() 
        plt.show()


    def get_rookie_season(self, player_id:int) -> str:
        career = playercareerstats.PlayerCareerStats(player_id=player_id)
        career_df = career.get_data_frames()[0]
        if career_df.empty:
            return get_current_season()
        rookie_season = career_df.iloc[0]['SEASON_ID']
        return rookie_season

    def get_team_roster_per_season(self, teamAbbreviation:str, season:str = get_current_season()) -> pd.DataFrame:
        team_details = teams.find_team_by_abbreviation(teamAbbreviation)
        team_id = team_details["id"]
        roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
        roster_data = roster.get_data_frames()[0]
        current_season = get_current_season()
        
        rookie_seasons = []
        for _, row in roster_data.iterrows():
            if row['EXP'] == 'R':
                rookie_season = current_season
            else: 
                rookie_season = self.get_rookie_season(row['PLAYER_ID'])
            rookie_seasons.append(rookie_season)
            time.sleep(0.6)  # To avoid hitting rate limits

        roster_data['ROOKIE_SEASON'] = rookie_seasons

        roster_data.drop(columns=['TeamID', 'SEASON', 'LeagueID', 'EXP', 'AGE', 'NUM', 'HOW_ACQUIRED', 'PLAYER_SLUG', 'NICKNAME'], inplace=True, axis=1)
        roster_data.set_index('PLAYER_ID', drop=True, inplace=True)
        return roster_data
    
    def get_player_teams(self, player_id:int) -> pd.DataFrame:
        career = playercareerstats.PlayerCareerStats(player_id=player_id)
        career_df = career.get_data_frames()[0]
        teams_played_for = career_df[['TEAM_ID', 'PLAYER_ID', 'SEASON_ID']].drop_duplicates().reset_index(drop=True)
        return teams_played_for
    
    

In [ ]:
lebron_player = players.find_players_by_full_name('Nikola Jokic')[0]
player_id = lebron_player['id']

player().get_alltime_player_stats(player_id=player_id)
#player().get_player_teams(player_id=1642365)
#player().get_player_info(player_id=player_id)

#player().get_current_season_stats(player_id=player_id)
#player().plot_stat_over_career(player_id=player_id, stat='PTS')

#player().get_team_roster_per_season(teamAbbreviation='SAS')

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203999,2015-16,00,1610612743,DEN,21.0,80,55,1733,307,600,0.512,28,84,0.333,154,190,0.811,181,379,560,189,79,50,104,208,796
1,203999,2016-17,00,1610612743,DEN,22.0,73,59,2038,494,854,0.578,45,139,0.324,188,228,0.825,212,506,718,359,61,55,171,214,1221
2,203999,2017-18,00,1610612743,DEN,23.0,75,73,2443,504,1010,0.499,111,280,0.396,266,313,0.850,195,608,803,458,90,61,210,212,1385
3,203999,2018-19,00,1610612743,DEN,24.0,80,80,2504,616,1206,0.511,83,270,0.307,289,352,0.821,228,637,865,580,108,55,248,228,1604
4,203999,2019-20,00,1610612743,DEN,25.0,73,73,2335,565,1071,0.528,80,255,0.314,246,301,0.817,166,545,711,512,85,44,226,222,1456
5,203999,2020-21,00,1610612743,DEN,26.0,72,72,2488,732,1293,0.566,92,237,0.388,342,394,0.868,205,575,780,599,95,48,222,192,1898
6,203999,2021-22,00,1610612743,DEN,27.0,74,74,2476,764,1311,0.583,97,288,0.337,379,468,0.810,206,813,1019,584,109,63,281,191,2004
7,203999,2022-23,00,1610612743,DEN,28.0,69,69,2323,646,1022,0.632,57,149,0.383,341,415,0.822,167,650,817,678,87,47,247,174,1690
8,203999,2023-24,00,1610612743,DEN,29.0,79,79,2737,822,1411,0.583,83,231,0.359,358,438,0.817,223,753,976,708,108,68,237,194,2085
9,203999,2024-25,00,1610612743,DEN,30.0,70,70,2571,786,1364,0.576,138,331,0.417,361,451,0.800,200,692,892,716,127,45,230,160,2071


In [ ]:
lebron_player = players.find_players_by_full_name('Nikola Jokic')[0]
player_id = lebron_player['id']

current_season_stats = player().get_alltime_player_stats(player_id=player_id)
current_season_stats = pd.DataFrame([current_season_stats.iloc[-1].to_dict()])


current_season_stats

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203999,2025-26,00,1610612743,DEN,30.0,25,25,874,268,437,0.613,55,127,0.433,155,184,0.842,75,234,309,271,35,19,85,73,746


In [ ]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
import pandas as pd

# Buscar jugador por nombre
player_dict = players.find_players_by_full_name("LeBron James")[0]
player_id = player_dict['id']

# Obtener estadísticas de carrera
career = playercareerstats.PlayerCareerStats(player_id=player_id)
df = career.get_data_frames()[0]

# Mostrar estadísticas por temporada
df

df.iloc[0]['SEASON_ID']

'2003-04'

In [ ]:
career.get_json()

'{"resource": "playercareerstats", "parameters": {"PerMode": "Totals", "PlayerID": 2544, "LeagueID": "00"}, "resultSets": [{"name": "SeasonTotalsRegularSeason", "headers": ["PLAYER_ID", "SEASON_ID", "LEAGUE_ID", "TEAM_ID", "TEAM_ABBREVIATION", "PLAYER_AGE", "GP", "GS", "MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB", "DREB", "REB", "AST", "STL", "BLK", "TOV", "PF", "PTS"], "rowSet": [[2544, "2003-04", "00", 1610612739, "CLE", 19.0, 79, 79, 3120, 622, 1492, 0.417, 63, 217, 0.29, 347, 460, 0.754, 99, 333, 432, 465, 130, 58, 273, 149, 1654], [2544, "2004-05", "00", 1610612739, "CLE", 20.0, 80, 80, 3388, 795, 1684, 0.472, 108, 308, 0.351, 477, 636, 0.75, 111, 477, 588, 577, 177, 52, 262, 146, 2175], [2544, "2005-06", "00", 1610612739, "CLE", 21.0, 79, 79, 3361, 875, 1823, 0.48, 127, 379, 0.335, 601, 814, 0.738, 75, 481, 556, 521, 123, 66, 260, 181, 2478], [2544, "2006-07", "00", 1610612739, "CLE", 22.0, 78, 78, 3190, 772, 1621, 0.476, 99, 310, 0.319

In [ ]:
df.columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [ ]:
eastern_conference = {
    'ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DET', 'IND',
    'MIA', 'MIL', 'NYK', 'ORL', 'PHI', 'TOR', 'WAS'
}

western_conference = {
    'DAL', 'DEN', 'GSW', 'HOU', 'LAC', 'LAL', 'MEM', 'MIN',
    'NOP', 'OKC', 'PHX', 'POR', 'SAC', 'SAS', 'UTA'
}

teams_list = list(eastern_conference.union(western_conference))
teams_list

['MIL',
 'ATL',
 'HOU',
 'GSW',
 'DAL',
 'CHA',
 'NYK',
 'ORL',
 'PHI',
 'SAC',
 'DET',
 'PHX',
 'WAS',
 'NOP',
 'BOS',
 'MEM',
 'IND',
 'SAS',
 'MIA',
 'LAC',
 'BKN',
 'POR',
 'TOR',
 'DEN',
 'LAL',
 'MIN',
 'CHI',
 'UTA',
 'CLE',
 'OKC']

In [ ]:
import time

def get_rookie_season(player_id:int) -> str:
    career = playercareerstats.PlayerCareerStats(player_id=player_id)
    career_df = career.get_data_frames()[0]
    if career_df.empty:
        return get_current_season()
    rookie_season = career_df.iloc[0]['SEASON_ID']
    return rookie_season

def get_team_roster_per_season(teamAbbreviation:str, season:str) -> pd.DataFrame:
    team_details = teams.find_team_by_abbreviation(teamAbbreviation)
    team_id = team_details["id"]
    roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
    roster_data = roster.get_data_frames()[0]
    
    rookie_seasons = []
    for _, row in roster_data.iterrows():
        rookie_season = get_rookie_season(row['PLAYER_ID'])
        rookie_seasons.append(rookie_season)
        time.sleep(0.6)  # To avoid hitting rate limits

    roster_data['ROOKIE_SEASON'] = rookie_seasons

    roster_data.drop(columns=['TeamID', 'SEASON', 'LeagueID', 'EXP', 'AGE', 'NUM', 'HOW_ACQUIRED', 'PLAYER_SLUG'], inplace=True, axis=1)
    return roster_data

In [ ]:
den_roster_2023 = get_team_roster_per_season("DEN", "2025-26")
den_roster_2023

,PLAYER,NICKNAME,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,SCHOOL,PLAYER_ID,ROOKIE_SEASON
0,Christian Braun,Christian,G,6-6,220,"APR 17, 2001",Kansas,1631128,2022-23
1,Curtis Jones,Curtis,G,6-3,195,"OCT 04, 2001",Iowa State,1642938,2025-26
2,Julian Strawther,Julian,G,6-6,205,"APR 18, 2002",Gonzaga,1631124,2023-24
3,Hunter Tyson,Hunter,F,6-8,215,"JUN 13, 2000",Clemson,1641816,2023-24
4,Tamar Bates,Tamar,G,6-4,195,"FEB 21, 2003",Missouri,1642926,2025-26
5,Peyton Watson,Peyton,G,6-8,200,"SEP 11, 2002",UCLA,1631212,2022-23
6,Tim Hardaway Jr.,Tim,G-F,6-5,205,"MAR 16, 1992",Michigan,203501,2013-14
7,Bruce Brown,Bruce,G-F,6-4,202,"AUG 15, 1996",Miami,1628971,2018-19
8,DaRon Holmes II,DaRon,F,6-9,225,"AUG 15, 2002",Dayton,1641747,2025-26
9,Nikola Jokić,Nikola,C,6-11,284,"FEB 19, 1995",Mega Basket,203999,2015-16


In [ ]:
den_roster_2023.set_index('PLAYER_ID', drop=True)

,PLAYER,NICKNAME,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,SCHOOL,ROOKIE_SEASON
PLAYER_ID,,,,,,,,
1631128,Christian Braun,Christian,G,6-6,220,"APR 17, 2001",Kansas,2022-23
1642938,Curtis Jones,Curtis,G,6-3,195,"OCT 04, 2001",Iowa State,2025-26
1631124,Julian Strawther,Julian,G,6-6,205,"APR 18, 2002",Gonzaga,2023-24
1641816,Hunter Tyson,Hunter,F,6-8,215,"JUN 13, 2000",Clemson,2023-24
1642926,Tamar Bates,Tamar,G,6-4,195,"FEB 21, 2003",Missouri,2025-26
1631212,Peyton Watson,Peyton,G,6-8,200,"SEP 11, 2002",UCLA,2022-23
203501,Tim Hardaway Jr.,Tim,G-F,6-5,205,"MAR 16, 1992",Michigan,2013-14
1628971,Bruce Brown,Bruce,G-F,6-4,202,"AUG 15, 1996",Miami,2018-19
1641747,DaRon Holmes II,DaRon,F,6-9,225,"AUG 15, 2002",Dayton,2025-26


In [ ]:
LAL_roster = get_team_roster_per_season("LAL",'2025-26')
LAL_roster

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612747,2025,00,Adou Thiero,Adou,adou-thiero,1,G,6-7,220,"MAY 08, 2004",21.0,R,Arkansas,1642876,Draft Rights Traded from BKN on 07/06/25
1,1610612747,2025,00,Jarred Vanderbilt,Jarred,jarred-vanderbilt,2,F,6-8,214,"APR 03, 1999",26.0,7,Kentucky,1629020,Traded from UTA on 02/09/23
2,1610612747,2025,00,Dalton Knecht,Dalton,dalton-knecht,4,F,6-6,215,"APR 19, 2001",24.0,1,Tennessee,1642261,#17 Pick in 2024 Draft
3,1610612747,2025,00,Deandre Ayton,Deandre,deandre-ayton,5,C,7-0,252,"JUL 23, 1998",27.0,7,Arizona,1629028,Signed on 07/06/25
4,1610612747,2025,00,Gabe Vincent,Gabe,gabe-vincent,7,G,6-2,200,"JUN 14, 1996",29.0,6,California-Santa Barbara,1629216,Signed on 07/06/23
5,1610612747,2025,00,Bronny James,Bronny,bronny-james,9,G,6-2,210,"OCT 06, 2004",21.0,1,Southern California,1642355,#55 Pick in 2024 Draft
6,1610612747,2025,00,Jaxson Hayes,Jaxson,jaxson-hayes,11,C-F,7-0,220,"MAY 23, 2000",25.0,6,Texas,1629637,Signed on 07/06/23
7,1610612747,2025,00,Jake LaRavia,Jake,jake-laravia,12,F,6-7,235,"NOV 03, 2001",24.0,3,Wake Forest,1631222,Signed on 07/06/25
8,1610612747,2025,00,Maxi Kleber,Maxi,maxi-kleber,14,F,6-10,240,"JAN 29, 1992",33.0,8,Bayern Munich,1628467,Traded from DAL on 02/02/25
9,1610612747,2025,00,Austin Reaves,Austin,austin-reaves,15,G,6-5,197,"MAY 29, 1998",27.0,4,Oklahoma,1630559,Signed on 08/03/21


In [ ]:
LAL_roster

[1629020,
 1642261,
 1629216,
 1642355,
 1631132,
 1629637,
 1628467,
 1630559,
 1627827,
 1629003,
 2544,
 203458,
 1629060,
 1630692,
 1641998,
 1629029,
 202693]

In [ ]:
players = [2544, 203458]
columns = ['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE', 'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']
empty_df = pd.DataFrame(columns=columns)


for player_id in players:
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    player_info_dict = player_info.get_normalized_dict()
    player_common_info = player_info_dict['CommonPlayerInfo'][0]

    empty_df.loc[len(empty_df)] = player_common_info


In [ ]:
empty_df

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,ROSTERSTATUS,GAMES_PLAYED_CURRENT_SEASON_FLAG,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,2544,LeBron,James,LeBron James,"James, LeBron",L. James,lebron-james,1984-12-30T00:00:00,St. Vincent-St. Mary HS (OH),USA,St. Vincent-St. Mary HS (OH)/USA,6-9,250,22,23,Forward,Active,Y,1610612747,Lakers,LAL,lakers,Los Angeles,lebron_james,2003,2025,N,Y,Y,2003,1,1,Y
1,203458,Alex,Len,Alex Len,"Len, Alex",A. Len,alex-len,1993-06-16T00:00:00,Maryland,Ukraine,Maryland/Ukraine,7-0,250,13,27,Center,Inactive,Y,1610612747,Lakers,LAL,lakers,Los Angeles,alex_len,2013,2024,N,Y,Y,2013,1,5,N


In [ ]:
import time 
def get_roster_data(roster):
    columns = ['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE', 'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']
    empty_df = pd.DataFrame(columns=columns)
    
    for player_id in roster:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        player_info_dict = player_info.get_normalized_dict()
        player_common_info = player_info_dict['CommonPlayerInfo'][0]
    
        empty_df.loc[len(empty_df)] = player_common_info

        time.sleep(0.8)
        

    return empty_df

In [ ]:
LAL_current_roster = get_roster_data(LAL_roster)

In [ ]:
LAL_current_roster.columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS',
       'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG'],
      dtype='object')

In [ ]:
LAL_current_roster

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,LAST_AFFILIATION,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,ROSTERSTATUS,GAMES_PLAYED_CURRENT_SEASON_FLAG,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_CITY,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,1629020,Jarred,Vanderbilt,Jarred Vanderbilt,"Vanderbilt, Jarred",J. Vanderbilt,jarred-vanderbilt,1999-04-03T00:00:00,Kentucky,USA,Kentucky/USA,6-8,214,7,2,Forward,Active,Y,1610612747,Lakers,LAL,lakers,Los Angeles,jarred_vanderbilt,2018,2025,Y,Y,Y,2018,2,41,N
1,1642261,Dalton,Knecht,Dalton Knecht,"Knecht, Dalton",D. Knecht,dalton-knecht,2001-04-19T00:00:00,Tennessee,USA,Tennessee/USA,6-6,215,1,4,Forward,Active,Y,1610612747,Lakers,LAL,lakers,Los Angeles,dalton_knecht,2024,2025,N,Y,Y,2024,1,17,N
2,1629216,Gabe,Vincent,Gabe Vincent,"Vincent, Gabe",G. Vincent,gabe-vincent,1996-06-14T00:00:00,California-Santa Barbara,USA,California-Santa Barbara/USA,6-2,200,6,7,Guard,Active,Y,1610612747,Lakers,LAL,lakers,Los Angeles,gabe_vincent,2019,2025,Y,Y,Y,Undrafted,Undrafted,Undrafted,N
3,1642355,Bronny,James,Bronny James,"James, Bronny",B. James,bronny-james,2004-10-06T00:00:00,Southern California,USA,Southern California/USA,6-2,210,1,9,Guard,Active,Y,1610612747,Lakers,LAL,lakers,Los Angeles,tmp_bronny_james,2024,2025,Y,Y,Y,2024,2,55,N
4,1631132,Christian,Koloko,Christian Koloko,"Koloko, Christian",C. Koloko,christian-koloko,2000-06-20T00:00:00,Arizona,Cameroon,Arizona/Cameroon,6-11,225,3,,Center,Inactive,Y,0,,,,,christian_koloko,2022,2025,Y,Y,Y,2022,2,33,N
5,1629637,Jaxson,Hayes,Jaxson Hayes,"Hayes, Jaxson",J. Hayes,jaxson-hayes,2000-05-23T00:00:00,Texas,USA,Texas/USA,7-0,220,6,11,Center-Forward,Active,Y,1610612747,Lakers,LAL,lakers,Los Angeles,jaxson_hayes,2019,2025,Y,Y,Y,2019,1,8,N
6,1628467,Maxi,Kleber,Maxi Kleber,"Kleber, Maxi",M. Kleber,maxi-kleber,1992-01-29T00:00:00,Bayern Munich,Germany,Bayern Munich/Germany,6-10,240,8,14,Forward,Active,Y,1610612747,Lakers,LAL,lakers,Los Angeles,maxi_kleber,2017,2025,N,Y,Y,Undrafted,Undrafted,Undrafted,N
7,1630559,Austin,Reaves,Austin Reaves,"Reaves, Austin",A. Reaves,austin-reaves,1998-05-29T00:00:00,Oklahoma,USA,Oklahoma/USA,6-5,197,4,15,Guard,Active,Y,1610612747,Lakers,LAL,lakers,Los Angeles,austin_reaves,2021,2025,N,Y,Y,Undrafted,Undrafted,Undrafted,N
8,1627827,Dorian,Finney-Smith,Dorian Finney-Smith,"Finney-Smith, Dorian",D. Finney-Smith,dorian-finney-smith,1993-05-04T00:00:00,Florida,USA,Florida/USA,6-7,220,9,2,Forward,Active,N,1610612745,Rockets,HOU,rockets,Houston,dorian_finney-smith,2016,2025,N,Y,Y,Undrafted,Undrafted,Undrafted,N
9,1629003,Shake,Milton,Shake Milton,"Milton, Shake",S. Milton,shake-milton,1996-09-26T00:00:00,Southern Methodist,USA,Southern Methodist/USA,6-5,205,8,20,Guard,Inactive,N,1610612747,Lakers,LAL,lakers,Los Angeles,shake_milton,2018,2024,Y,Y,Y,2018,2,54,N


In [ ]:
LAL_current_roster['PLAYERCODE']

0       jarred_vanderbilt
1           dalton_knecht
2            gabe_vincent
3        tmp_bronny_james
4        christian_koloko
5            jaxson_hayes
6             maxi_kleber
7           austin_reaves
8     dorian_finney-smith
9            shake_milton
10           lebron_james
11               alex_len
12          rui_hachimura
13         jordan_goodwin
14           trey_jemison
15               luka_don
16        markieff_morris
Name: PLAYERCODE, dtype: object

In [ ]:
drop = ["DISPLAY_LAST_COMMA_FIRST",'LAST_AFFILIATION',"ROSTERSTATUS", "GAMES_PLAYED_CURRENT_SEASON_FLAG","TEAM_NAME","TEAM_ABBREVIATION","TEAM_CODE","TEAM_CITY","PLAYERCODE","DLEAGUE_FLAG","NBA_FLAG","GAMES_PLAYED_FLAG","DRAFT_ROUND","GREATEST_75_FLAG"]

In [ ]:
LAL_current_roster.drop(columns=drop, inplace=True)

In [ ]:
LAL_current_roster.head()

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,1629020,Jarred,Vanderbilt,Jarred Vanderbilt,J. Vanderbilt,jarred-vanderbilt,1999-04-03T00:00:00,Kentucky,USA,6-8,214,7,2,Forward,1610612747,2018,2025,2018,41
1,1642261,Dalton,Knecht,Dalton Knecht,D. Knecht,dalton-knecht,2001-04-19T00:00:00,Tennessee,USA,6-6,215,1,4,Forward,1610612747,2024,2025,2024,17
2,1629216,Gabe,Vincent,Gabe Vincent,G. Vincent,gabe-vincent,1996-06-14T00:00:00,California-Santa Barbara,USA,6-2,200,6,7,Guard,1610612747,2019,2025,Undrafted,Undrafted
3,1642355,Bronny,James,Bronny James,B. James,bronny-james,2004-10-06T00:00:00,Southern California,USA,6-2,210,1,9,Guard,1610612747,2024,2025,2024,55
4,1631132,Christian,Koloko,Christian Koloko,C. Koloko,christian-koloko,2000-06-20T00:00:00,Arizona,Cameroon,6-11,225,3,,Center,0,2022,2025,2022,33


In [ ]:
LAL_current_roster.columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY',
       'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'TEAM_ID',
       'FROM_YEAR', 'TO_YEAR', 'DRAFT_YEAR', 'DRAFT_NUMBER'],
      dtype='object')

In [ ]:
def get_roster_stats_by_team(teamAbrev):
    drop = ["PLAYER_SLUG","DISPLAY_FI_LAST","DISPLAY_LAST_COMMA_FIRST",'LAST_AFFILIATION',"ROSTERSTATUS", "GAMES_PLAYED_CURRENT_SEASON_FLAG","TEAM_NAME","TEAM_ABBREVIATION","TEAM_CODE","TEAM_CITY","PLAYERCODE","DLEAGUE_FLAG","NBA_FLAG","GAMES_PLAYED_FLAG","DRAFT_ROUND","GREATEST_75_FLAG"]
    team_roster = get_team_roster_per_season(teamAbrev,'2024-25')["PLAYER_ID"].tolist()
    team_current_roster = get_roster_data(team_roster)
    team_current_roster.drop(columns=drop, inplace=True)
    
    team_current_roster.to_json(f'{teamAbrev}_roster.json', orient = 'split', compression = 'infer', index = 'true')

    return team_current_roster
    
    

In [ ]:
get_roster_stats_by_team("LAL")

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,1629020,Jarred,Vanderbilt,Jarred Vanderbilt,1999-04-03T00:00:00,Kentucky,USA,6-8,214,7,2,Forward,1610612747,2018,2025,2018,41
1,1642261,Dalton,Knecht,Dalton Knecht,2001-04-19T00:00:00,Tennessee,USA,6-6,215,1,4,Forward,1610612747,2024,2025,2024,17
2,1629216,Gabe,Vincent,Gabe Vincent,1996-06-14T00:00:00,California-Santa Barbara,USA,6-2,200,6,7,Guard,1610612747,2019,2025,Undrafted,Undrafted
3,1642355,Bronny,James,Bronny James,2004-10-06T00:00:00,Southern California,USA,6-2,210,1,9,Guard,1610612747,2024,2025,2024,55
4,1631132,Christian,Koloko,Christian Koloko,2000-06-20T00:00:00,Arizona,Cameroon,6-11,225,3,,Center,0,2022,2025,2022,33
5,1629637,Jaxson,Hayes,Jaxson Hayes,2000-05-23T00:00:00,Texas,USA,7-0,220,6,11,Center-Forward,1610612747,2019,2025,2019,8
6,1628467,Maxi,Kleber,Maxi Kleber,1992-01-29T00:00:00,Bayern Munich,Germany,6-10,240,8,14,Forward,1610612747,2017,2025,Undrafted,Undrafted
7,1630559,Austin,Reaves,Austin Reaves,1998-05-29T00:00:00,Oklahoma,USA,6-5,197,4,15,Guard,1610612747,2021,2025,Undrafted,Undrafted
8,1627827,Dorian,Finney-Smith,Dorian Finney-Smith,1993-05-04T00:00:00,Florida,USA,6-7,220,9,2,Forward,1610612745,2016,2025,Undrafted,Undrafted
9,1629003,Shake,Milton,Shake Milton,1996-09-26T00:00:00,Southern Methodist,USA,6-5,205,8,20,Guard,1610612747,2018,2024,2018,54


In [ ]:
get_roster_stats_by_team("GSW")

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,1627780,Gary,Payton II,Gary Payton II,1992-12-01T00:00:00,Oregon State,USA,6-2,195,9,0,Guard,1610612744,2016,2025,Undrafted,Undrafted
1,1630228,Jonathan,Kuminga,Jonathan Kuminga,2002-10-06T00:00:00,NBA G League Ignite,DRC,6-7,225,4,1,Forward,1610612744,2021,2025,2021,7
2,1642379,Taran,Armstrong,Taran Armstrong,2002-01-15T00:00:00,"Cairns, Australia",Australia,6-5,190,1,1,Guard,1610612744,2024,2024,Undrafted,Undrafted
3,1641764,Brandin,Podziemski,Brandin Podziemski,2003-02-25T00:00:00,Santa Clara,USA,6-4,205,2,2,Guard,1610612744,2023,2025,2023,19
4,1630541,Moses,Moody,Moses Moody,2002-05-31T00:00:00,Arkansas,USA,6-5,211,4,4,Guard,1610612744,2021,2025,2021,14
5,1626172,Kevon,Looney,Kevon Looney,1996-02-06T00:00:00,UCLA,USA,6-9,222,10,55,Forward,1610612740,2015,2025,2015,30
6,1627741,Buddy,Hield,Buddy Hield,1992-12-17T00:00:00,Oklahoma,Bahamas,6-4,220,9,7,Guard,1610612744,2016,2025,2016,6
7,202710,Jimmy,Butler III,Jimmy Butler III,1989-09-14T00:00:00,Marquette,USA,6-6,230,14,10,Forward,1610612744,2011,2025,2011,30
8,1630296,Braxton,Key,Braxton Key,1997-02-14T00:00:00,Virginia,USA,6-8,225,5,12,Forward,1610612744,2021,2024,Undrafted,Undrafted
9,1630611,Gui,Santos,Gui Santos,2002-06-22T00:00:00,Minas,Brazil,6-7,185,2,15,Forward,1610612744,2023,2025,2022,55


In [ ]:
get_roster_stats_by_team("DEN")

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,1631128,Christian,Braun,Christian Braun,2001-04-17T00:00:00,Kansas,USA,6-6,220,3,0,Guard,1610612743,2022,2025,2022,21
1,1629008,Michael,Porter Jr.,Michael Porter Jr.,1998-06-29T00:00:00,Missouri,USA,6-10,218,6,17,Forward,1610612751,2018,2025,2018,14
2,1631124,Julian,Strawther,Julian Strawther,2002-04-18T00:00:00,Gonzaga,USA,6-6,205,2,3,Guard,1610612743,2023,2025,2023,29
3,201566,Russell,Westbrook,Russell Westbrook,1988-11-12T00:00:00,UCLA,USA,6-4,200,17,18,Guard,1610612758,2008,2025,2008,4
4,1641816,Hunter,Tyson,Hunter Tyson,2000-06-13T00:00:00,Clemson,USA,6-8,215,2,5,Forward,1610612743,2023,2025,2023,37
5,201599,DeAndre,Jordan,DeAndre Jordan,1988-07-21T00:00:00,Texas A&M,USA,6-11,265,17,9,Center,1610612740,2008,2025,2008,35
6,1631212,Peyton,Watson,Peyton Watson,2002-09-11T00:00:00,UCLA,USA,6-8,200,3,8,Guard,1610612743,2022,2025,2022,30
7,203967,Dario,Šarić,Dario Šarić,1994-04-08T00:00:00,Anadolu Efes,Croatia,6-10,225,9,20,Forward-Center,1610612758,2016,2025,2014,12
8,1641790,PJ,Hall,PJ Hall,2002-02-21T00:00:00,Clemson,USA,6-8,245,1,,Center,0,2024,2025,Undrafted,Undrafted
9,1641747,DaRon,Holmes II,DaRon Holmes II,2002-08-15T00:00:00,Dayton,USA,6-9,225,0,14,Forward,1610612743,2024,2025,2024,22


In [ ]:
get_roster_stats_by_team("DAL")

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,203957,Danté,Exum,Danté Exum,1995-07-13T00:00:00,Australian Institute of Sport,Australia,6-5,214,9,0,Guard,1610612742,2014,2025,2014,5
1,1631108,Max,Christie,Max Christie,2003-02-10T00:00:00,Michigan State,USA,6-5,190,3,00,Guard,1610612742,2022,2025,2022,35
2,1630702,Jaden,Hardy,Jaden Hardy,2002-07-05T00:00:00,NBA G League Ignite,USA,6-3,198,3,1,Guard,1610612742,2021,2025,2022,37
3,1641726,Dereck,Lively II,Dereck Lively II,2004-02-12T00:00:00,Duke,USA,7-1,230,2,2,Center,1610612742,2023,2025,2023,12
4,203076,Anthony,Davis,Anthony Davis,1993-03-11T00:00:00,Kentucky,USA,6-10,253,13,3,Forward-Center,1610612742,2012,2025,2012,1
5,203939,Dwight,Powell,Dwight Powell,1991-07-20T00:00:00,Stanford,Canada,6-10,240,11,7,Forward-Center,1610612742,2014,2025,2014,45
6,1641765,Olivier-Maxence,Prosper,Olivier-Maxence Prosper,2002-07-03T00:00:00,Marquette,Canada,6-7,230,2,18,Forward,1610612763,2023,2025,2023,24
7,1630314,Brandon,Williams,Brandon Williams,1999-11-22T00:00:00,Arizona,USA,6-1,190,3,10,Guard,1610612742,2021,2025,Undrafted,Undrafted
8,202681,Kyrie,Irving,Kyrie Irving,1992-03-23T00:00:00,Duke,Australia,6-2,195,14,11,Guard,1610612742,2011,2025,2011,1
9,1630230,Naji,Marshall,Naji Marshall,1998-01-24T00:00:00,Xavier,USA,6-6,220,5,13,Forward,1610612742,2020,2025,Undrafted,Undrafted
